In [1]:
!pip install transformers

In [2]:
from transformers import DistilBertTokenizer, DistilBertModel
import torch
from sklearn.metrics.pairwise import cosine_similarity

# Load DistilBERT tokenizer and model
model_name = 'distilbert-base-uncased'
tokenizer = DistilBertTokenizer.from_pretrained(model_name)
model = DistilBertModel.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

In [4]:
import pandas as pd
import re
import torch
from sklearn.metrics.pairwise import cosine_similarity

In [5]:
def DescriptionCleaning(df, columns, name):
  columns_list = []
  columns_list.append(columns)
  columns_list.append(name)
  print(columns_list)
  df = df[columns_list]
  df=df.dropna()

  labelRaw = list(df[columns])
  label = []
  for text in labelRaw:
    text = str(text)
    temp = re.sub('[^a-zA-Z]+', ' ', text)
    temp = temp.lower().strip()
    label.append(temp)

  df[columns]=label

  return df

In [6]:
file_path = "/content/Accidents_data_Processed.csv"
df = pd.read_csv(file_path)
df = DescriptionCleaning(df, "Description", "BUSINESS NAME")

['Description', 'BUSINESS NAME']


In [7]:
file_path = "/content/Image to text output.xlsx"
manual_driving_df = pd.read_excel(file_path)
df1 = DescriptionCleaning(manual_driving_df, "Llava output", "Video name")

['Llava output', 'Video name']


In [9]:
count = 0
data = []
obj = {}
# Creating Empty DataFrame and Storing it in variable df
outputDF = pd.DataFrame()
for i in range(len(df)):
    for j in range(len(df1)):
        obj = {"business_name": "", "prompt1": "", "video_name": "", "prompt2": "", "score": 0}
        row1 = df.iloc[i]
        row2 = df1.iloc[j]
        if i != j:
            prompt1 = row1['Description']
            prompt2 = row2['Llava output']

            # Tokenize and encode sentences
            inputs1 = tokenizer(prompt1, return_tensors="pt", padding=True, truncation=True)
            inputs2 = tokenizer(prompt2, return_tensors="pt", padding=True, truncation=True)
            # Obtain model embeddings
            with torch.no_grad():
                embeddings1 = model(**inputs1).last_hidden_state.mean(dim=1)
                embeddings2 = model(**inputs2).last_hidden_state.mean(dim=1)
            # Calculate cosine similarity
            similarity_matrix = cosine_similarity(embeddings1, embeddings2)
            similarity_score = similarity_matrix[0][0]
            obj["business_name"] = row1["BUSINESS NAME"]
            obj["prompt1"] = prompt1
            obj["video_name"] = row2["Video name"]
            obj["prompt2"] = prompt2
            obj["score"] = similarity_score
            data.append(obj)
            if similarity_score > 0.95:
              count = count + 1
              #print(f"Similarity between Prompt {i+1} and Prompt {j+1}: {similarity_score}")
              #print(prompt1)
              #print(prompt2)
              print(i+1)

print(count)

7
9
9
9
9
9
11
19
49
53
66
103
110
113
115
153
206
215
224
233
237
252
266
289
291
302
310
312
312
312
312
312
312
312
346
373
36


In [10]:
outputDF = pd.DataFrame(data)
outputDF.to_excel('Distil_BERT_similarity_score.xlsx', sheet_name = 'LLava_comparision')

In [ ]:
df1 = DescriptionCleaning(manual_driving_df, "VQA and LLaMA Output", "Video name")
count = 0
data = []
obj = {}
# Creating Empty DataFrame and Storing it in variable df
outputDF = pd.DataFrame()
for i in range(len(df)):
    for j in range(len(df1)):
        obj = {"business_name": "", "prompt1": "", "video_name": "", "prompt2": "", "score": 0}
        row1 = df.iloc[i]
        row2 = df1.iloc[j]
        if i != j:
            prompt1 = row1['Description']
            prompt2 = row2['VQA and LLaMA Output']

            # Tokenize and encode sentences
            inputs1 = tokenizer(prompt1, return_tensors="pt", padding=True, truncation=True)
            inputs2 = tokenizer(prompt2, return_tensors="pt", padding=True, truncation=True)
            # Obtain model embeddings
            with torch.no_grad():
                embeddings1 = model(**inputs1).last_hidden_state.mean(dim=1)
                embeddings2 = model(**inputs2).last_hidden_state.mean(dim=1)
            # Calculate cosine similarity
            similarity_matrix = cosine_similarity(embeddings1, embeddings2)
            similarity_score = similarity_matrix[0][0]
            obj["business_name"] = row1["BUSINESS NAME"]
            obj["prompt1"] = prompt1
            obj["video_name"] = row2["Video name"]
            obj["prompt2"] = prompt2
            obj["score"] = similarity_score
            data.append(obj)
            if similarity_score > 0.95:
              count = count + 1
              #print(f"Similarity between Prompt {i+1} and Prompt {j+1}: {similarity_score}")
              #print(prompt1)
              #print(prompt2)
              print(i+1)

print(count)

['VQA and LLaMA Output', 'Video name']
15
18
33
66
89
89
89
103
103
103
113
159
159
159
176
176
197
206
206
206
206
206
206
206
206
206
206
206
224
224
224
224
227
228
260
280
280
280
280
285
291
291
291
298
299
304
304
304
309
309
309
309
310
310
310
310
312
312
312
312
312
312
312
312
312
312
314
367
381


In [ ]:
outputDF = pd.DataFrame(data)

In [ ]:
import pandas as pd
import numpy as np
from openpyxl import load_workbook

path = r"/content/Distil_BERT_similarity_score.xlsx"

book = load_workbook(path)
writer = pd.ExcelWriter(path, engine = 'openpyxl')
writer.book = book

outputDF.to_excel(writer, sheet_name = 'VQA_and_LLama')
writer.close()

In [ ]:
df1 = DescriptionCleaning(manual_driving_df, "BLIPVQA 13B and LLaMA Output", "Video name")
count = 0
data = []
obj = {}
# Creating Empty DataFrame and Storing it in variable df
outputDF = pd.DataFrame()
for i in range(len(df)):
    for j in range(len(df1)):
        obj = {"business_name": "", "prompt1": "", "video_name": "", "prompt2": "", "score": 0}
        row1 = df.iloc[i]
        row2 = df1.iloc[j]
        if i != j:
            prompt1 = row1['Description']
            prompt2 = row2['BLIPVQA 13B and LLaMA Output']

            # Tokenize and encode sentences
            inputs1 = tokenizer(prompt1, return_tensors="pt", padding=True, truncation=True)
            inputs2 = tokenizer(prompt2, return_tensors="pt", padding=True, truncation=True)
            # Obtain model embeddings
            with torch.no_grad():
                embeddings1 = model(**inputs1).last_hidden_state.mean(dim=1)
                embeddings2 = model(**inputs2).last_hidden_state.mean(dim=1)
            # Calculate cosine similarity
            similarity_matrix = cosine_similarity(embeddings1, embeddings2)
            similarity_score = similarity_matrix[0][0]
            obj["business_name"] = row1["BUSINESS NAME"]
            obj["prompt1"] = prompt1
            obj["video_name"] = row2["Video name"]
            obj["prompt2"] = prompt2
            obj["score"] = similarity_score
            data.append(obj)
            if similarity_score > 0.95:
              count = count + 1
              #print(f"Similarity between Prompt {i+1} and Prompt {j+1}: {similarity_score}")
              #print(prompt1)
              #print(prompt2)
              print(i+1)

print(count)

In [ ]:
outputDF = pd.DataFrame(data)

In [ ]:
import pandas as pd
import numpy as np
from openpyxl import load_workbook

path = r"/content/Distil_BERT_similarity_score.xlsx"

book = load_workbook(path)
writer = pd.ExcelWriter(path, engine = 'openpyxl')
writer.book = book

outputDF.to_excel(writer, sheet_name = 'BLIPVQA_13B_and_LLaMA')
writer.close()